# Statement

The image we are going to process is a two-color interference image with red and black phases. Therefore, I firstly perform binarization processing on it to convert the color three-channel image into a two-channel grayscale image.

<p float="left">
  <img src="./image/orginal_image.png" width="45%" height=""/>
  <img src="./image/result.png" width="45%" />
</p>

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# import finder

ModuleNotFoundError: No module named 'finder'

# Pre-processing

- Read image file as a openCV format - BGR
- Convert BGR image to 1 channel grayscale
- Applies Gaussian smoothing with a 5×5 kernel to reduce high-frequency noise
- Binarizes the image using a fixed threshold (threshold_val=40)
- Performs opening to remove small white pepper noise
- Figuring labels connected regions in the binary image and filtering "pepper noise" again 

_NOTE: This recognition algorithm is mainly based on image recognition. Of course, it can be modified for the recognition of interference rings in real-time video_

Read the image and convert it into a channel of grayscale image for later processing firstly. Then, perform Gaussian blur processing on it to initially reduce noise interference.

In [ ]:
#Reading and preprocessing
img = cv2.imread("test/img.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)

## Thresholding

- **Threshold value:** 40

- **Max value:** 255 (pixels above the threshold are set to this value)

- **Method:** cv2.THRESH_BINARY (standard binary thresholding)

After multiple rounds of empirical tuning, I determined that a threshold value of 40 provides the best balance.
This setting allows the bright and dark fringes in the interference pattern to remain clearly visible, while significantly suppressing background noise and artifacts. As a result, the binary image retains essential structural features without being overwhelmed by minor disturbances, making it an effective preprocessing step for downstream morphological and contour-based operations.

In [ ]:
#Thresholding
threshold_val = 40
_, binary = cv2.threshold(blur, threshold_val, 255, cv2.THRESH_BINARY)

### Morphological Opening

To remove small isolated white noise from the binary image, I applied a **morphological opening** operation, which consists of an erosion followed by a dilation.

- **Kernel shape**: 4×4 rectangular structuring element  
- **Purpose**: Effectively eliminates small white spots and smooths the object contours  
- This step enhances the clarity of the target fringe regions by cleaning up minor foreground noise, preparing the image for more accurate component analysis.


In [ ]:
#Open operation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

### Connected Component Filtering

After morphological cleaning, I performed **connected component analysis** to identify and isolate meaningful regions based on area.

- **connectedComponentsWithStats** provides detailed statistics (`area`, `bounding box`, etc.) for each connected region.
- I skipped the background (`i = 0`) and filtered components by **area size** between **300 and 2800** pixels.
- This filtering helps exclude both small noise blobs and overly large irrelevant regions, retaining only target-like features in the image.
- The result is a clean binary mask (`clean`) containing candidate areas suitable for further processing such as center detection.


In [ ]:
#clean
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened, connectivity=8)
clean = np.zeros_like(gray)

for i in range(1, num_labels):
    area = stats[i, cv2.CC_STAT_AREA]
    if 300 < area < 2800:
        clean[labels == i] = 255

### Circle Center Detection via Voting

To locate the center of the circular interference pattern, I used a custom **voting-based center detection** algorithm:

- **Input**: `clean` — the binary image after noise removal and region filtering  
- **Parameter**: `100` — the radius or neighborhood range used for center voting  
- **Method**: This function scans the bright regions and allows each pixel to "vote" for possible circle centers within a defined radius. The point with the highest votes is considered the most probable center.

This method is robust to slight distortions and noise, and works well for detecting approximate geometric centers in fringe-like patterns.


In [ ]:
x0, y0 = finder.find_circle_center_voting(clean,100)

### Visualizing the Detected Center

To visualize the detected circle center on the original image:

- `img.copy()` creates a copy to avoid modifying the original image.  
- `cv2.circle` draws a filled circle (`-1` thickness) of radius 5 pixels at the detected center `(x0, y0)` in **magenta** (BGR: (255, 0, 255)).  
- `cv2.imshow` opens a window to display the image with the marked center.  
- `cv2.waitKey(0)` waits indefinitely for a key press to proceed.  
- `cv2.destroyAllWindows()` closes the display window properly.

This step helps intuitively verify the accuracy of the center detection algorithm.


In [ ]:
output = img.copy()
cv2.circle(output, (int(x0), int(y0)), 5, (255, 0, 255), -1)

cv2.imshow("Center Result", output)
cv2.waitKey(0)
cv2.destroyAllWindows()